In [ ]:
import pandas as pd
from pathlib import Path
# import os
# rootpath=os.getcwd()
# rootpath=Path(r'D:\repos\spark\data')
# rootpath=Path(r'D:\repos\spark\data')
rootpath=Path(r'C:\Users\crowl\OneDrive\Education\C04443 Master of Artificial Intelligence\2024s4 - 43008 Reinforcement Learning\Assignment 3 - Group - SPARK\data')
rootpath=Path(r'C:\Users\crowl\OneDrive\Education\C04443 Master of Artificial Intelligence\2024s4 - 43008 Reinforcement Learning\Assignment 3 - Group - SPARK\data')


customers      = pd.read_csv(rootpath.joinpath('olist_customers_dataset.csv'))
# geolocation    = pd.read_csv(rootpath.joinpath('olist_geolocation_dataset.csv'))
orders         = pd.read_csv(rootpath.joinpath('olist_orders_dataset.csv'))
order_items    = pd.read_csv(rootpath.joinpath('olist_order_items_dataset.csv'))
# order_payments = pd.read_csv(rootpath.joinpath('olist_order_payments_dataset.csv'))
order_reviews  = pd.read_csv(rootpath.joinpath('olist_order_reviews_dataset.csv'))
products       = pd.read_csv(rootpath.joinpath('olist_products_dataset.csv'))
# sellers        = pd.read_csv(rootpath.joinpath('olist_sellers_dataset.csv'))
product_category_name_translation = pd.read_csv(rootpath.joinpath('product_category_name_translation.csv'))

products = products[['product_id','product_category_name']]
products=(products
          .merge(product_category_name_translation,how='left',on='product_category_name')
          .drop(columns=['product_category_name'])
          .rename(columns={'product_category_name_english':'product_category'})
          )


orders = (orders[['order_id', 'customer_id', 'order_purchase_timestamp']]
          .rename(columns={'order_purchase_timestamp':'timestamp'})
          )

# Merge customers and orders tables together as customer_id and order_id is 1:1
customers_orders=(customers
                  .rename(columns={'customer_zip_code_prefix':'zip_code',
                                   'customer_city':'city',
                                   'customer_state':'state'})
                  .merge(orders, how='inner', on='customer_id')
            )


# De-duplicate customer attributes, will be used later
customers_unique = (customers_orders[['customer_unique_id','customer_id','zip_code','city','state']]
                    .drop_duplicates(subset='customer_unique_id', keep='first')
                    .drop(columns=['customer_id'])
                    .sort_values(by=['customer_unique_id'])
                    .reset_index(drop=True)
                    # .reset_index()
                    )

# Aggregate order_items to find the value for each customer_id/order_id
order_value = (order_items
               .groupby('order_id')
               .agg({'product_id':'count', 'price':'sum'})
               .rename(columns={'product_id':'product_count', 'price':'value'})
               .merge(customers_orders[['order_id','customer_unique_id']], how='inner', on='order_id')
               .sort_values(by='product_count',ascending=False)
               .reset_index()
               )

# Further aggregate order_value to customer_unique_id level
customer_value = (order_value
                  .groupby('customer_unique_id')
                  .agg({'order_id':'count', 'product_count':'sum', 'value':'sum'})
                  .rename(columns={'order_id':'order_count'})
                  .merge(customers_unique, how='inner', on='customer_unique_id')
                  .sort_values(by=['product_count'],ascending=False)
                  .reset_index(drop=True)
                  .reset_index()
                  .rename(columns={'index':'customer_num_id'})
                  )

# Identify the top X customers to reduce the dataset to
top_customers = (customer_value
                 .head(50)
                 )

out_customers =(top_customers
                .drop(columns=['customer_unique_id', 'order_count', 'product_count','value'])
    )

# 
top_order_items = (order_items[['order_id', 'order_item_id', 'product_id', 'price']]
                   .merge(customers_orders[['timestamp','order_id','customer_unique_id']], how='inner', on='order_id')
                   .merge(top_customers[['customer_unique_id','customer_num_id']], how='inner', on='customer_unique_id')
                   .merge(products, how='left', on='product_id')
                   .drop(columns='customer_unique_id')
                   .sort_values(by=['timestamp','order_id'])
                   .reset_index(drop=True)
                   .reset_index()
                   .rename(columns={'index':'order_num_id'})
                   )


top_order_items['timestamp'] = pd.to_datetime(top_order_items['timestamp']) + pd.to_timedelta(top_order_items['order_item_id'], unit='s')
top_order_items['interaction_id'] = 'order-'+ top_order_items['order_num_id'].astype(str)
top_order_items['review_score'] = 0
top_order_items['type'] = 'buy'


products_unique = (top_order_items[['product_id','product_category','price']]
                   .drop_duplicates(subset='product_id', keep='first')
                   .fillna(value={'product_category':'shrubbery'})
                   .sort_values(by=['product_category'])
                   .reset_index(drop=True)
                   .reset_index()
                   .rename(columns={'index':'product_num_id'})
                   )

products_unique['product_category_id'] = products_unique.groupby(['product_category']).cumcount()+1
products_unique['product_name_id'] = products_unique['product_category'] + '-' + products_unique['product_category_id'].astype(str)

category_unique=(products_unique[['product_category']]
                .drop_duplicates(subset='product_category', keep='first')
                .reset_index(drop=True)
                .reset_index()
                .rename(columns={'index':'category_num_id'})
    )

out_products = (products_unique
                .drop(columns=['product_id'])
                .merge(category_unique, how='left', on='product_category')
                )


top_order_items=(top_order_items
                 .drop(columns=['product_category', 'price'])
                 .merge(products_unique, how='left', on='product_id')
    )

unique_order_products=(top_order_items[['order_id','order_item_id','product_num_id','customer_num_id']]
                       .drop_duplicates(subset=['order_id','product_num_id'], keep='first')
    )


top_reviews =(order_reviews[['review_id','order_id','review_score','review_answer_timestamp']]
              .rename(columns={'review_answer_timestamp':'timestamp'})
              .merge(unique_order_products, how='inner', on='order_id')
              .reset_index(drop=True)
              .reset_index()
              .rename(columns={'index':'review_num_id'})
    )

top_reviews['timestamp'] = pd.to_datetime(top_reviews['timestamp']) + pd.to_timedelta(top_reviews['order_item_id'], unit='s')
top_reviews['interaction_id'] = 'review-'+ top_reviews['review_num_id'].astype(str)
top_reviews['type'] = 'rate'
top_reviews['value'] = 1

out_interactions=(pd.concat([top_order_items[['timestamp','interaction_id','product_num_id','customer_num_id','review_score', 'type', 'price']].rename(columns={'price':'value'}),
                        top_reviews[['timestamp','interaction_id','product_num_id','customer_num_id','review_score', 'type', 'value']].rename(columns={'price':'value'})
                        ])
                  .sort_values(by=['timestamp','interaction_id'])
                  )
